# Running multiple calculations on a given model


## Aim

This notebook shows how we can run multiple calculations of a given structure

### Setup

The initial setup is very similar to the other tutorials, such as `singlepoint.ipynb`, which goes into more detail about what each step is doing

Load the aiida profile and code:

In [ ]:
from aiida import load_profile
load_profile()

In [ ]:
from aiida.orm import load_code
code = load_code("janus@localhost")

Inputs should include the model, code, metadata, and any other keyword arguments expected by the calculation we are running:

In [ ]:
from aiida.orm import Str, Float, Bool
inputs = {
    "code": code,
    "model": model,
    "arch": Str(model.architecture),
    "precision": Str("float64"),
    "device": Str("cpu"),
    "fmax": Float(0.1), 
    "opt_cell_lengths": Bool(False), 
    "opt_cell_fully": Bool(True), 
    "metadata": {"options": {"resources": {"num_machines": 1}}},
    }

We must now choose the calculations to perform:

In [ ]:
from aiida.plugins import CalculationFactory
geomoptCalc = CalculationFactory("mlip.opt")
descriptorsCalc = CalculationFactory("mlip.descriptors")

Now we can create our WorkGraph. This includes passing in the inputs, checking the amount of structures we have and interating through. Note for this workbook we have decreased the amount of structures to two for the sake of simplicity. In the loop we can call each structure; run geomopt calculation, pass the output file into descriptors calculation, and get the final outputs of all the structures

In [ ]:
from aiida_workgraph import WorkGraph
from aiida.orm import StructureData
from ase.build import bulk
from ase.io import read

wg = WorkGraph("Calculation Workgraph")
wg.inputs = inputs

num_structs = len(read("../../examples/tutorials/structures/lj-traj.xyz", index=":"))

for i in range(num_structs - 9):

    structure = StructureData(ase=read("../../examples/tutorials/structures/lj-traj.xyz", index=f"{i}"))
    
    geomopt_calc = wg.add_task(
        geomoptCalc,
        code = wg.inputs.code,
        model = wg.inputs.model,
        arch = wg.inputs.arch,
        precision = wg.inputs.precision,
        device = wg.inputs.device,
        metadata = wg.inputs.metadata,
        fmax = wg.inputs.fmax,
        opt_cell_lengths = wg.inputs.opt_cell_lengths,
        opt_cell_fully = wg.inputs.opt_cell_fully,
        struct = structure,
    )

    descriptors_calc = wg.add_task(
        descriptorsCalc,
        code = wg.inputs.code,
        model = wg.inputs.model,
        arch = wg.inputs.arch,
        precision = wg.inputs.precision,
        device = wg.inputs.device,
        metadata = wg.inputs.metadata,
        struct = geomopt_calc.outputs.final_structure,
        calc_per_element = True
    )

    wg.outputs.final_structures = (
        {f"structs{i}" : descriptors_calc.outputs.xyz_output}
        )
    


In [ ]:
wg

Now we can run the calculations

In [ ]:
wg.run()

In order to pass the outputs from the WorkGraph into sample_split.py we create a list of structures

In [ ]:
import shutil, os
from pathlib import Path

descriptors_output = list(wg.outputs.final_structures)
traj_structs = []

for i in range(len(descriptors_output)):

    # create list variable which holds output structs
    singlefiledata = wg.outputs.final_structures[f"structs{i}"]
    traj_structs.append(singlefiledata.value)


print(traj_structs)

In [ ]:
Now we can run the split script

In [ ]:
from sample_split import process_and_split_data


process_and_split_data(
    trajectory_path = traj_structs,
    config_types = "",
    n_samples = 2,
    prefix = "",
    scale = 1.0e5,
    append_mode = False
    )
